In [28]:
from langchain.memory import ConversationSummaryMemory


/Users/jeyeolbaek/Developer/study/fullstack_gpt/env/lib/python3.9/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/jeyeolbaek/Developer/study/fullstack_gpt/env/lib/python3.9/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_kwargs={'frequency_penalty': 0, 'logit_bias': {}, 'max_tokens': 100, 'n': 1, 'presence_penalty': 0, 'request_timeout': None, 'temperature': 0.1, 'top_p': 1})